<a href="https://colab.research.google.com/github/Viktor193/innopolis/blob/main/%22%D0%9F%D1%80%D0%BE%D0%BC%D0%B5%D0%B6%D1%83%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B0%D1%82%D1%82%D0%B5%D1%81%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D0%91%D0%BB%D0%BE%D0%BA_3_%D0%9D%D0%B8%D0%BA%D0%B8%D1%82%D0%B5%D0%BD%D0%BA%D0%BE_%D0%92_%D0%92_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 198 kB 65.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=80e813f5753a61bc5a8254fccab21ceea8e52dde0fdf236d26af622b646653fd
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
# загружаем библиотеки
import pandas as pd
import numpy as np

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession #to define a SparkSession

sc = SparkContext('local') #spark_connection
spark = SparkSession(sc) # open spark session

## Задание. Обучите модель классификации для цветков Iris'а

---
Примерная последовательность действий:
1. Взять данные iris.csv
2. Загрузить в pyspark
3. При помощи VectorAssembler преобразовать все колонки с признаками в одну
(использовать PipeLine)
4. Разбить данные на train и test
5. Создать модель логистической регресии и обучить ее
6. Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test
множестве

---

### 1-2. Загружаем iris через pyspark

In [4]:
# Используем options() чтобы задать несколько опций:
# header=True - Для чтения использует первую строку в качестве имен столбцов
# sep="," - Устанавливает разделитель для каждого поля и значения
# quote="" - Нужно установить пустую строку для отключения кавычки, тк quote по-умолчанию ", тк ковычки не являются разделителем
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True, quote="").csv('iris.csv')

In [6]:
df.show(10)

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|             .2| ""Setosa"""|
|         "4.9|              3|             1.4|             .2| ""Setosa"""|
|         "4.7|            3.2|             1.3|             .2| ""Setosa"""|
|         "4.6|            3.1|             1.5|             .2| ""Setosa"""|
|           "5|            3.6|             1.4|             .2| ""Setosa"""|
|         "5.4|            3.9|             1.7|             .4| ""Setosa"""|
|         "4.6|            3.4|             1.4|             .3| ""Setosa"""|
|           "5|            3.4|             1.5|             .2| ""Setosa"""|
|         "4.4|            2.9|             1.4|             .2| ""Setosa"""|
|         "4.9|            3.1|             1.5|             .1|

In [8]:
#преобразуем названия колонок, создав новый датафрейм
newColumns = ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_new = df.toDF(*newColumns)

#удалим двойные ковычки в первом и последнем атрибутах
df_new = df_new.withColumn("sepal_length", regexp_replace(col("sepal_length"), '"', ""))\
    .withColumn("variety", regexp_replace(col("variety"), '"', ""))

#преобразуем в другие типы данных
df_new = df_new \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

In [9]:
df_new.show(10) #посмотрим на преобразование

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 10 rows



In [10]:
df_new.printSchema() #посмотрим на типы атрибутов

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



In [12]:
df_new.summary().show()


+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    25%|               5.1|                2.8|               1.6|               0.3|     null|
|    50%|               5.8|                3.0|               4.3|               1.3|     null|
|    75%|               6.4|                3.3|               5.1|               1.8|     null|
|    max|               7.9|  

### 3. При помощи VectorAssembler преобразовать все колонки с признаками в одну (использовать PipeLine)

In [13]:
pipeline = Pipeline(stages = 
[
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features'),
  StringIndexer(inputCol='variety', outputCol='varietyInd')
]
)

In [14]:
# подгоняем через заданный pipeline наш датасет
pipelineTrained = pipeline.fit(df_new)

In [15]:
# применяем метод transform для объекта VectorAssembler (преобразуем признаки в вектор) 
pipelineTrained.transform(df_new).show()

+------------+-----------+------------+-----------+-------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|         Features|varietyInd|
+------------+-----------+------------+-----------+-------+-----------------+----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|       0.0|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|       0.0|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|       0.0|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|       0.0|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|       0.0|
|         5.4|        3.9|         1.7|        0.4| Setosa|[5.4,3.9,1.7,0.4]|       0.0|
|         4.6|        3.4|         1.4|        0.3| Setosa|[4.6,3.4,1.4,0.3]|       0.0|
|         5.0|        3.4|         1.5|        0.2| Setosa|[5.0,3.4,1.5,0.2]|       0.0|
|         4.4|       

In [18]:
# с новым признаком Features создаем новый датафрейм
df_features = pipelineTrained.transform(df_new)
df_features.show()

+------------+-----------+------------+-----------+-------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|         Features|varietyInd|
+------------+-----------+------------+-----------+-------+-----------------+----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|       0.0|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|       0.0|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|       0.0|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|       0.0|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|       0.0|
|         5.4|        3.9|         1.7|        0.4| Setosa|[5.4,3.9,1.7,0.4]|       0.0|
|         4.6|        3.4|         1.4|        0.3| Setosa|[4.6,3.4,1.4,0.3]|       0.0|
|         5.0|        3.4|         1.5|        0.2| Setosa|[5.0,3.4,1.5,0.2]|       0.0|
|         4.4|       

### 4. Разбить данные на train и test


In [33]:
# назначаем соответствующие веса и указываем seed (рандомное разделение ввыборки сохранилось)
train, test = df_features.randomSplit([0.7, 0.3], seed=12345)

In [34]:
# посмотрим на обучающую выборку
train.show()

+------------+-----------+------------+-----------+----------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|         Features|varietyInd|
+------------+-----------+------------+-----------+----------+-----------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|[4.3,3.0,1.1,0.1]|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|[4.4,2.9,1.4,0.2]|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|[4.4,3.0,1.3,0.2]|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|[4.4,3.2,1.3,0.2]|       0.0|
|         4.5|        2.3|         1.3|        0.3|    Setosa|[4.5,2.3,1.3,0.3]|       0.0|
|         4.6|        3.1|         1.5|        0.2|    Setosa|[4.6,3.1,1.5,0.2]|       0.0|
|         4.6|        3.4|         1.4|        0.3|    Setosa|[4.6,3.4,1.4,0.3]|       0.0|
|         4.6|        3.6|         1.0|        0.2|    Setosa|[4.6,3.6,1.0,0.2]|

In [35]:
test.show()

+------------+-----------+------------+-----------+----------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|         Features|varietyInd|
+------------+-----------+------------+-----------+----------+-----------------+----------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|[4.6,3.2,1.4,0.2]|       0.0|
|         4.8|        3.1|         1.6|        0.2|    Setosa|[4.8,3.1,1.6,0.2]|       0.0|
|         4.9|        2.5|         4.5|        1.7| Virginica|[4.9,2.5,4.5,1.7]|       2.0|
|         5.0|        3.0|         1.6|        0.2|    Setosa|[5.0,3.0,1.6,0.2]|       0.0|
|         5.0|        3.2|         1.2|        0.2|    Setosa|[5.0,3.2,1.2,0.2]|       0.0|
|         5.0|        3.5|         1.3|        0.3|    Setosa|[5.0,3.5,1.3,0.3]|       0.0|
|         5.1|        3.5|         1.4|        0.3|    Setosa|[5.1,3.5,1.4,0.3]|       0.0|
|         5.4|        3.4|         1.5|        0.4|    Setosa|[5.4,3.4,1.5,0.4]|

In [36]:
test.count()


34

In [37]:
train.count()

116

### 5. Создать модель логистической регресии и обучить ее

Теперь воспользуемся логистической регрессией (Logistic Regression), которая есть в PySpark, в качестве алгоритма Machine learning. Для этого нужно указать признаки, на которых модель обучается, и признак, который нужно классифицировать. Мы преобразовали признаки цветка в вектор под названием features, поэтому именно его и указываем в аргументе, а целевым признаком будет числовое поле varietyInd

In [38]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

Осталось только получить предсказания. Для этого вызывается метод transform, который принимает тестовую выборку, а также обучающую (как обучилась модель):

In [45]:
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [46]:
# посмотрим, какие предсказания дает модель на обучающей выборке
train_res.show()

+------------+-----------+------------+-----------+----------+-----------------+----------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|         Features|varietyInd|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+-----------------+----------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|[4.3,3.0,1.1,0.1]|       0.0|[56.6274526968210...|[1.0,3.2319469090...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|[4.4,2.9,1.4,0.2]|       0.0|[47.4233614265561...|[1.0,3.6362613229...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|[4.4,3.0,1.3,0.2]|       0.0|[51.4018089744364...|[1.0,1.8249111068...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|[4.4,3.2,1.3,0.2]|       0.0|[57.2610708901335...|[1.0,5.4124305806.

In [47]:
# посмотрим, какие предсказания дает модель на тестовой выборке
test_res.show()

+------------+-----------+------------+-----------+----------+-----------------+----------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|         Features|varietyInd|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+-----------------+----------+--------------------+--------------------+----------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|[4.6,3.2,1.4,0.2]|       0.0|[54.5148862173048...|[1.0,2.6331753744...|       0.0|
|         4.8|        3.1|         1.6|        0.2|    Setosa|[4.8,3.1,1.6,0.2]|       0.0|[47.7902539965958...|[1.0,2.5525872921...|       0.0|
|         4.9|        2.5|         4.5|        1.7| Virginica|[4.9,2.5,4.5,1.7]|       2.0|[-35.241794416653...|[4.48835247452736...|       2.0|
|         5.0|        3.0|         1.6|        0.2|    Setosa|[5.0,3.0,1.6,0.2]|       0.0|[43.1632549559503...|[1.0,2.1013773728.

### 6. Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве

Для оценки качества предсказания в spark реализованно несколько классов Если мы решаем задачу бинарной классификации (то есть классов - 2), то нам подойдет BinaryCLassificationEvaluator, а если классов больше 2-х, то MulticlassClassificationEvaluator.

Проверим эффективность модели, используя метрику качества MulticlassClassificationEvaluator, тк признак variety состоит из 3х значений. Нужно лишь указать целевой признак ***varietyInd***, а затем вызвать метод ***evaluate*** и передать в него наши предсказания

In [53]:
ev = MulticlassClassificationEvaluator(labelCol='varietyInd')

In [54]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(train_res)*100, '%')

Точность предсказания на обучающей выборке: 98.27586206896552 %


In [55]:
print('Точность предсказания на тестовой выборке:', ev.evaluate(test_res)*100, '%')

Точность предсказания на тестовой выборке: 100.0 %


### Выводы:
to do